In [58]:
import base64

import requests

from lxml import html as lhtml

from tqdm.notebook import tqdm

In [59]:
documents = []

for page_i in tqdm(range(1, 101)):
    page = requests.post('https://hightech.fm/wp-admin/admin-ajax.php', data={
        'action': 'loadmore',
        'posts_per_page': 7,
        'template': 'other',
        'offset': 0,
        'page': page_i,
        'nonce': 'c13ec0ef9f',
    })
    
    if not page.ok:
        print(page.status_code, page_i)
    
    tree = lhtml.fromstring(page.text)
    urls = tree.xpath('//article/a[@href]/attribute::href')
    
    if not urls:
        break
    
    documents.extend(urls)
    
documents[:10]

['https://hightech.fm/2020/04/01/seastar-cd2',
 'https://hightech.fm/2020/04/01/sarge',
 'https://hightech.fm/2020/04/01/biomars',
 'https://hightech.fm/2020/04/01/extreme-high-frequency-enable',
 'https://hightech.fm/2020/04/01/nightingale',
 'https://hightech.fm/2020/04/01/hubble-new-type',
 'https://hightech.fm/2020/04/01/uran-sattelites',
 'https://hightech.fm/2020/04/01/3d-print-covid-19',
 'https://hightech.fm/2020/04/01/prod',
 'https://hightech.fm/2020/04/01/mars-bees']

In [60]:
len(documents), len(set(documents))

(700, 700)

In [74]:
def process_page(url):
    page = requests.get(url)
    tree = lhtml.fromstring(page.content)

    title = tree.xpath('//h1[@itemprop="headline"]/text()')
    title = ''.join(title) or None

    description = tree.xpath('//div[@class="page-content__head"]//p[contains(@class, "lead")]//text()')
    description = ''.join(description) or None

    rubric = tree.xpath('//div[@class="hero-main__category"]//span[@itemprop="genre"]/text()')

    info = {
        'title': title,
        'description': description,
        'rubric': rubric,
        'url': page.url,
    }

    return info

In [75]:
process_page('https://hightech.fm/2020/03/30/coronavirus-terminal')

{'title': 'Робототехническая компания представила терминалы для проверки на коронавирус',
 'description': 'Производитель сервисных роботов — компания Promobot — представила терминалы для проверки на коронавирус. Устройства предназначены для дистанционного измерения температуры у людей в местах повышенной проходимости.',
 'rubric': ['COVID-19'],
 'url': 'https://hightech.fm/2020/03/30/coronavirus-terminal'}

In [76]:
records = []

for doc in documents[:10]:
    record = process_page(doc)
    records.append(record)

In [79]:
import json

with open('/tmp/text.txt', 'w') as f_json:
    for record in records:
        record_str = json.dumps(record, ensure_ascii=False)
        print(record_str, file=f_json)

In [80]:
!cat /tmp/text.txt

{"title": "Dornier Seawings испытала самолет-амфибию с тандемным расположением двигателей", "description": "Dornier Seawings испытала самолет-амфибию с тандемным расположением двигателей. Испытания аппарата под названием Seastar CD2 были признаны успешными, пишет FlightGlobal.", "rubric": ["Новости"], "url": "https://hightech.fm/2020/04/01/seastar-cd2"}
{"title": "ВВС США будут использовать геофизические ракеты для гиперзвуковых испытаний", "description": "ВВС США будут использовать малозаметные ракеты для гиперзвуковых испытаний. Американские военные заключили контракт с компанией Exos Aerospace Systems & Technologies на доработку геофизических ракет SARGE для их использования в рамках создания гиперзвукового оружия.", "rubric": ["Новости"], "url": "https://hightech.fm/2020/04/01/sarge"}
{"title": "Новые бактериальные батарейки позволят производить энергию на Марсе", "description": "Новые бактериальные батарейки позволят производить энергию на Марсе. Биогенератор электричества на осно